# 코루틴을 동시에 실행하는 방법들.
Python의 `asyncio` 라이브러리를 사용하면 코루틴을 동시에 실행할 수 있습니다. 다양한 방법이 있지만, 여기서는 가장 일반적으로 사용되는 몇 가지 방법을 소개하겠습니다:

### 1. `asyncio.gather()`

`asyncio.gather()`는 여러 코루틴을 동시에 실행하고 그 결과를 모아서 반환합니다. (gather()는 코루틴들이 완료될 때까지 기다린다.)

```python
import asyncio

async def task1():
    await asyncio.sleep(1)
    return "result1"

async def task2():
    await asyncio.sleep(2)
    return "result2"

async def main():
    results = await asyncio.gather(task1(), task2())
    print(results)  # ['result1', 'result2']

asyncio.run(main())
```

### 2. `asyncio.create_task()`

`asyncio.create_task()`를 사용하여 코루틴을 백그라운드 태스크로 시작할 수 있습니다. 반환된 태스크 객체를 사용하여 나중에 결과를 가져올 수 있습니다.

```python
import asyncio

async def task1():
    await asyncio.sleep(1)
    return "result1"

async def task2():
    await asyncio.sleep(2)
    return "result2"

async def main():
    t1 = asyncio.create_task(task1())
    t2 = asyncio.create_task(task2())
    
    print(await t1)  # result1
    print(await t2)  # result2

asyncio.run(main())
```

### 3. `asyncio.wait()`

`asyncio.wait()`는 여러 코루틴을 동시에 실행하되, 완료된 코루틴의 결과를 즉시 반환하거나 특정 조건이 충족될 때까지 기다릴 수 있습니다.

```python
import asyncio

async def task1():
    await asyncio.sleep(1)
    return "result1"

async def task2():
    await asyncio.sleep(2)
    return "result2"

async def main():
    done, pending = await asyncio.wait([task1(), task2()], return_when=asyncio.FIRST_COMPLETED)
    
    for task in done:
        print(task.result())

asyncio.run(main())
```

이 코드는 `task1()`이나 `task2()` 중 하나가 먼저 완료되면 해당 결과를 출력합니다.

### 주의사항

코루틴을 동시에 실행한다는 것은 실제로 병렬로 실행되는 것이 아니라, 비동기적으로 "동시에" 실행되는 것입니다. `asyncio`는 싱글 스레드에서 실행되므로, CPU-bound 작업을 병렬로 실행하려면 다른 방법을 사용해야 합니다(예: 멀티프로세싱 또는 멀티스레딩).

이러한 방법들을 사용하면 여러 코루틴을 동시에 실행할 수 있어 I/O-bound 작업, 특히 네트워크 요청과 같은 작업에 매우 유용합니다.

# 주의 깊게 살펴볼 코드.
```python
async def binance_client():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client, user_timeout=60)

    # 각 코인에 대한 소켓 생성
    price_tasks = [bm.trade_socket(f'{crypto}USDT') for crypto in cryptos]
    ticker_tasks = [bm.symbol_ticker_socket(f'{crypto}USDT') for crypto in cryptos]

    # 모든 코인의 소켓을 동시에 처리하기 위해 asyncio.gather 사용
    all_tasks = [handle_price_socket(crypto, task) for crypto, task in zip(cryptos, price_tasks)] + [handle_ticker_socket(crypto, task) for crypto, task in zip(cryptos, ticker_tasks)]
    await asyncio.gather(*all_tasks)
```
`create_task를 사용하지 않고 gather를 사용하는 이유:`<br>
이벤트 루프: asyncio.create_task()는 현재 실행 중인 이벤트 루프에서 새로운 태스크를 생성합니다.<br>
binance_client가 호출될 때 이벤트 루프가 실행 중인지 확인해야 합니다.<br>
만약 asyncio.run(binance_client())와 같은 방식으로 호출한다면, 이것이 문제가 될 수 있습니다.<br>
왜냐하면 asyncio.run()은 내부적으로 새로운 이벤트 루프를 시작하기 때문입니다.<br>

그렇다면 `asyncio.run(binance_client())와 같은 방식으로 호출한다면, 이것이 문제가 될 수 있습니다.` 이 말이 왜 문제가 되는지 알아볼까요?<br>
`asyncio.run()`과 `asyncio.create_task()` 사이의 상호작용에 대해 자세히 살펴보겠습니다.<br>
`asyncio.run()` 함수는 새로운 이벤트 루프를 시작하고, 주어진 코루틴을 해당 이벤트 루프에서 실행합니다. 이 이벤트 루프는 코루틴의 실행이 완료되면 종료됩니다.<br>
만약 `asyncio.run()` 안에서 `asyncio.create_task()`를 호출하게 되면, 현재 (즉, `asyncio.run()`에 의해 생성된) 이벤트 루프에서 태스크가 생성되며 문제가 없습니다.<br>
하지만 문제는 이렇게 생성된 태스크가 `asyncio.run()`이 종료되기 전에 완료되지 않을 경우입니다. `asyncio.run()`이 반환되면 이벤트 루프도 종료되므로, 아직 완료되지 않은 태스크들은 종료되지 않은 상태로 남게 됩니다.<br>
예를 들어:<br>

```python
import asyncio

async def my_coroutine():
    task = asyncio.create_task(some_long_running_task())
    # do other stuff
    await asyncio.sleep(1)  # Let's assume this is the end of our coroutine

asyncio.run(my_coroutine())
```

여기서 `some_long_running_task()`가 1초보다 오래 걸린다면, `asyncio.run(my_coroutine())`이 반환될 때 `some_long_running_task()`는 여전히 실행 중이 될 것입니다. 이것은 의도하지 않은 결과를 초래할 수 있습니다.<br>
따라서 일반적으로 `asyncio.run()`을 사용할 때는 주의가 필요하며, 내부에서 생성하는 모든 태스크가 종료되기 전에 `asyncio.run()`이 반환되지 않도록 해야 합니다.<br>
이를 방지하기 위한 한 가지 방법은 모든 태스크가 완료될 때까지 기다리는 것입니다. 예를 들어, `asyncio.gather()`를 사용하여 모든 태스크가 완료될 때까지 기다릴 수 있습니다.<br>

`따라서 create_task()를 사용할꺼면 아래와 같이 코딩 해야함.`
```python
tasks = []

async def binance_client():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client, user_timeout=60)

    # 각 코인에 대한 소켓 생성
    price_tasks = [bm.trade_socket(f'{crypto}USDT') for crypto in cryptos]
    ticker_tasks = [bm.symbol_ticker_socket(f'{crypto}USDT') for crypto in cryptos]

    # 모든 코루틴을 동시에 백그라운드에서 실행하기 위해 asyncio.create_task 사용
    for crypto, task in zip(cryptos, price_tasks):
        t = asyncio.create_task(handle_price_socket(crypto, task))
        tasks.append(t)
        
    for crypto, task in zip(cryptos, ticker_tasks):
        t = asyncio.create_task(handle_ticker_socket(crypto, task))
        tasks.append(t)
        
    # 만약 다른 작업이 없으면, 현재 생성된 태스크들이 종료될 때까지 대기
    await asyncio.gather(*tasks)
```